In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import *

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import matplotlib.pyplot as plt
import numpy as np

from resnet import *

import pickle
from collections import OrderedDict


/home/pratyush/miniconda3/envs/pytorch_3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import argparse
import os

# parser = argparse.ArgumentParser(description='PyTorch Cifar10 Training')
# parser.add_argument('--ngpu', type=int, default=1, help='0 = CPU.')
# parser.add_argument('--gpu_id', type=int, default=0,
#                     help='device range [0,ngpu-1]')


# args = parser.parse_args()

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
ngpu = 1
gpu_id = 1
if ngpu == 1:
    # make only devices indexed by #gpu_id visible
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.01

In [4]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net2 = ResNet18()
net2 = net2.to(device)
if device == 'cuda':
    net2 = torch.nn.DataParallel(net2)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net2.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


Model Importing

In [5]:
# Load ResNet18 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net_basic = ResNet18()
net_basic = net_basic.to(device)
if device == 'cuda':
    net_basic = torch.nn.DataParallel(net_basic)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_basic.load_state_dict(state_dict)

#Load ResNet18 model with FGSM adversarial training
print('==> Loading FGSM-trained model..')
net_fgsm = ResNet18()
net_fgsm = net_fgsm.to(device)
if device == 'cuda':
    net_fgsm = torch.nn.DataParallel(net_fgsm)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_fgsmTraining.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_fgsm.load_state_dict(state_dict)

#Load ResNet18 model with PGD adversarial training
print('==> Loading PGD-trained model..')
net_pgd = ResNet18()
net_pgd = net_pgd.to(device)
if device == 'cuda':
    net_pgd = torch.nn.DataParallel(net_pgd)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_githubPGD20_epoch200_better.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_pgd.load_state_dict(state_dict)

#Load ResNet18 model with Chaos Loss minimization training
print('==> Loading ChaosLoss Minimization model..')
net_chaosLoss = ResNet18()
net_chaosLoss = net_chaosLoss.to(device)
if device == 'cuda':
    net_chaosLoss = torch.nn.DataParallel(net_chaosLoss)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/new_loss_function/resnet18_cifar10_chaos_regularized_pgd_torchattacks_lambdaChaos0.8_epoch200_lr0.01_iters20.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('resnet18.'):
        name = k.replace('resnet18', 'module')
    else:
        name = k
    new_state_dict[name] = v

net_chaosLoss.load_state_dict(new_state_dict)


==> Loading basic ResNet model..
==> Loading FGSM-trained model..
==> Loading PGD-trained model..
==> Loading ChaosLoss Minimization model..


<All keys matched successfully>

In [6]:
# net_basic_copy1 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net_basic_copy1 = ResNet18()
net_basic_copy1 = net_basic_copy1.to(device)
if device == 'cuda':
    net_basic_copy1 = torch.nn.DataParallel(net_basic_copy1)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_basic_copy1.load_state_dict(state_dict)

==> Loading basic ResNet model..


<All keys matched successfully>

In [7]:
# net_basic_copy2 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net_basic_copy2 = ResNet18()
net_basic_copy2 = net_basic_copy2.to(device)
if device == 'cuda':
    net_basic_copy2 = torch.nn.DataParallel(net_basic_copy2)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_basic_copy2.load_state_dict(state_dict)

==> Loading basic ResNet model..


<All keys matched successfully>

In [8]:
# net_basic_copy3 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net_basic_copy3 = ResNet18()
net_basic_copy3 = net_basic_copy3.to(device)
if device == 'cuda':
    net_basic_copy3 = torch.nn.DataParallel(net_basic_copy3)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_basic_copy3.load_state_dict(state_dict)

==> Loading basic ResNet model..


<All keys matched successfully>

In [9]:
# net_basic_copy4 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net_basic_copy4 = ResNet18()
net_basic_copy4 = net_basic_copy4.to(device)
if device == 'cuda':
    net_basic_copy4 = torch.nn.DataParallel(net_basic_copy4)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_basic_copy4.load_state_dict(state_dict)

==> Loading basic ResNet model..


<All keys matched successfully>

In [10]:
# net_basic_copy5 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net_basic_copy5 = ResNet18()
net_basic_copy5 = net_basic_copy5.to(device)
if device == 'cuda':
    net_basic_copy5 = torch.nn.DataParallel(net_basic_copy5)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_basic_copy5.load_state_dict(state_dict)

==> Loading basic ResNet model..


<All keys matched successfully>

In [11]:
# net_basic_copy6 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net_basic_copy6 = ResNet18()
net_basic_copy6 = net_basic_copy6.to(device)
if device == 'cuda':
    net_basic_copy6 = torch.nn.DataParallel(net_basic_copy6)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_basic_copy6.load_state_dict(state_dict)

==> Loading basic ResNet model..


<All keys matched successfully>

Normal Testing

In [12]:
def test(epoch, net):

    '''
    This function evaluate net on test dataset
    '''

    global acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100 * correct / total
    return test_loss/len(testloader)

In [13]:
train_losses=[]
test_losses=[]
epochs=1

for epoch in range(0,epochs):
    # train_losses.append(train(epoch, net))
    test_losses.append(test(epoch, net_basic))
    scheduler.step()
    
print('net_basic on default dataset accuracy: %d %%' % (acc))

net_basic on default dataset accuracy: 89 %


/home/pratyush/miniconda3/envs/pytorch_3.9/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [13]:
train_losses=[]
test_losses=[]
epochs=1

for epoch in range(0,epochs):
    # train_losses.append(train(epoch, net))
    test_losses.append(test(epoch, net_chaosLoss))
    scheduler.step()
    
print('net_chaos100 on default dataset accuracy: %d %%' % (acc))

In [15]:
train_losses=[]
test_losses=[]
epochs=1

for epoch in range(0,epochs):
    # train_losses.append(train(epoch, net))
    test_losses.append(test(epoch, net_pgd))
    scheduler.step()
    
print('net_pgd on default dataset accuracy: %d %%' % (acc))

net_pgd on default dataset accuracy: 58 %


FGSM Testing

In [ ]:
def FGSM(net, x, y, eps):
        '''
        inputs:
            net: the network through which we pass the inputs
            x: the original example which we aim to perturb to make an adversarial example
            y: the true label of x
            eps: perturbation budget

        outputs:
            x_adv : the adversarial example constructed from x
            h_adv: output of the last softmax layer when applying net on x_adv 
            y_adv: predicted label for x_adv
            pert: perturbation applied to x (x_adv - x)
        '''

        x_ = Variable(x.data, requires_grad=True)
        h_ = net(x_)
        criterion= torch.nn.CrossEntropyLoss()
        cost = criterion(h_, y)
        net.zero_grad()
        cost.backward()

        #perturbation
        pert= eps*x_.grad.detach().sign()
        
        x_adv = x_ + pert

        h_adv = net(x_adv)
        _,y_adv=torch.max(h_adv.data,1)
        return x_adv, h_adv, y_adv, pert


In [ ]:
def test_fgsm(net, net_adv, eps):
    accuracy=0
    net.train()
    net_adv.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)

        x_adv, h_adv, y_adv, pert = FGSM (net, inputs, targets, eps)
            
        outputs = net_adv(x_adv)
        loss = criterion(outputs, targets)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [35]:
# accuracy=test_fgsm(net_basic_copy1, net_basic, 8/255)
# print("accuracy of net_basic on FGSM-attacked dataset:", accuracy)

In [36]:
# accuracy=test_fgsm(net_basic_copy2, net_fgsm, 8/255)
# print("accuracy of net_fgsm on FGSM-attacked dataset:", accuracy)

In [ ]:
accuracy=test_fgsm(net_basic_copy3, net_chaosLoss, 8/255)
print("accuracy of net_chaosLoss on FGSM-attacked dataset:", accuracy)

accuracy of net_chaosLoss on FGSM-attacked dataset: 86.82


PGD Testing

In [ ]:
# def PGD(net,x,y,alpha,epsilon,iter):
#     '''
#     inputs:
#         net: the network through which we pass the inputs
#         x: the original example which we aim to perturb to make an adversarial example
#         y: the true label of x
#         alpha: step size
#         epsilon: perturbation budget 
#         iter: number of iterations in the PGD algorithm

#     outputs:
#         x_adv : the adversarial example constructed from x
#         h_adv: output of the last softmax layer when applying net on x_adv 
#         y_adv: predicted label for x_adv
#         pert: perturbation applied to x (x_adv - x)
#     '''

#     delta = torch.zeros_like(x, requires_grad=True)
#     for i in range(iter):
#         criterion=nn.CrossEntropyLoss()
#         loss = criterion(net(x + delta), y)
#         loss.backward()
#         delta.data = (delta + x.shape[0]*alpha*delta.grad.data).clamp(-epsilon,epsilon)
#         delta.grad.zero_()
#     pert = delta.detach()
#     x_adv = x + pert
#     h_adv = net(x_adv)
#     _,y_adv = torch.max(h_adv.data,1)
#     return x_adv, h_adv, y_adv, pert

In [ ]:
# def test_PGD(net,net_pgd,alpha,eps,iter):
#     acc=0
#     net.train()
#     net_pgd.eval()
#     test_loss = 0
#     correct = 0
#     total = 0
#     for batch_idx, (inputs, targets) in enumerate(testloader):
#             inputs, targets = inputs.to(device), targets.to(device)
#             x_adv,_,_,_=PGD(net,inputs,targets,alpha,eps,iter)

#             with torch.no_grad():
#                 outputs = net_pgd(x_adv)
#                 loss = criterion(outputs, targets)

#                 test_loss += loss.item()
#                 _, predicted = outputs.max(1)
#                 total += targets.size(0)
#                 correct += predicted.eq(targets).sum().item()
#     acc = 100 * correct / total
#     return acc

In [ ]:
# alpha=3/255
# eps=8/255
# acc1_pgd=[]
# for iter in [3,7,12]:
#     acc=test_PGD(net_basic_copy1,net_basic,alpha,eps,iter)
#     print("accuracy of net_basic against PGD attack with iters=",iter,": ",acc)

accuracy of net_basic against PGD attack with iters= 3 :  83.12
accuracy of net_basic against PGD attack with iters= 7 :  78.6
accuracy of net_basic against PGD attack with iters= 12 :  75.09


In [ ]:
# alpha=3/255
# eps=8/255
# acc1_pgd=[]
# for iter in [3,7,12]:
#     acc=test_PGD(net_basic_copy2,net_fgsm,alpha,eps,iter)
#     print("accuracy of net_fgsm against PGD attack with iters=",iter,": ",acc)

accuracy of net_fgsm against PGD attack with iters= 3 :  88.64
accuracy of net_fgsm against PGD attack with iters= 7 :  88.36
accuracy of net_fgsm against PGD attack with iters= 12 :  88.23


In [ ]:
# alpha=3/255
# eps=8/255
# acc1_pgd=[]
# for iter in [3,7,12]:
#     acc=test_PGD(net_basic_copy2,net_pgd,alpha,eps,iter)
#     print("accuracy of net_pgd against PGD attack with iters=",iter,": ",acc)

PGD Testing new code

In [ ]:
# from numpy import copy
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# import torch.backends.cudnn as cudnn

# import torchvision
# import torchvision.transforms as transforms

# import os

# learning_rate = 0.1
# epsilon = 0.0314
# k = 7
# alpha = 0.00784
# file_name = 'pgd_adversarial_training'

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
# ])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
# ])

# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

# class LinfPGDAttack(object):
#     def __init__(self, model):
#         self.model = model

#     def perturb(self, x_natural, y):
#         x = x_natural.detach()
#         x = x + torch.zeros_like(x).uniform_(-epsilon, epsilon)
#         for i in range(k):
#             x.requires_grad_()
#             with torch.enable_grad():
#                 logits = self.model(x)
#                 loss = F.cross_entropy(logits, y)
#             grad = torch.autograd.grad(loss, [x])[0]
#             x = x.detach() + alpha * torch.sign(grad.detach())
#             x = torch.min(torch.max(x, x_natural - epsilon), x_natural + epsilon)
#             x = torch.clamp(x, 0, 1)
#         return x

# def attack(x, y, model, adversary):
#     model_copied = copy.deepcopy(model)
#     model_copied.eval()
#     adversary.model = model_copied
#     adv = adversary.perturb(x, y)
#     return adv

# net = net_basic

# adversary = LinfPGDAttack(net)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

# def train(epoch):
#     print('\n[ Train epoch: %d ]' % epoch)
#     net.train()
#     train_loss = 0
#     correct = 0
#     total = 0
#     for batch_idx, (inputs, targets) in enumerate(train_loader):
#         inputs, targets = inputs.to(device), targets.to(device)
#         optimizer.zero_grad()

#         adv = adversary.perturb(inputs, targets)
#         adv_outputs = net(adv)
#         loss = criterion(adv_outputs, targets)
#         loss.backward()

#         optimizer.step()
#         train_loss += loss.item()
#         _, predicted = adv_outputs.max(1)

#         total += targets.size(0)
#         correct += predicted.eq(targets).sum().item()
        
#         if batch_idx % 10 == 0:
#             print('\nCurrent batch:', str(batch_idx))
#             print('Current adversarial train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
#             print('Current adversarial train loss:', loss.item())

#     print('\nTotal adversarial train accuarcy:', 100. * correct / total)
#     print('Total adversarial train loss:', train_loss)

# def test(epoch):
#     print('\n[ Test epoch: %d ]' % epoch)
#     net.eval()
#     benign_loss = 0
#     adv_loss = 0
#     benign_correct = 0
#     adv_correct = 0
#     total = 0
#     with torch.no_grad():
#         for batch_idx, (inputs, targets) in enumerate(test_loader):
#             inputs, targets = inputs.to(device), targets.to(device)
#             total += targets.size(0)

#             outputs = net(inputs)
#             loss = criterion(outputs, targets)
#             benign_loss += loss.item()

#             _, predicted = outputs.max(1)
#             benign_correct += predicted.eq(targets).sum().item()

#             if batch_idx % 10 == 0:
#                 print('\nCurrent batch:', str(batch_idx))
#                 print('Current benign test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
#                 print('Current benign test loss:', loss.item())

#             adv = adversary.perturb(inputs, targets)
#             adv_outputs = net(adv)
#             loss = criterion(adv_outputs, targets)
#             adv_loss += loss.item()

#             _, predicted = adv_outputs.max(1)
#             adv_correct += predicted.eq(targets).sum().item()

#             if batch_idx % 10 == 0:
#                 print('Current adversarial test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
#                 print('Current adversarial test loss:', loss.item())

#     print('\nTotal benign test accuarcy:', 100. * benign_correct / total)
#     print('Total adversarial test Accuarcy:', 100. * adv_correct / total)
#     print('Total benign test loss:', benign_loss)
#     print('Total adversarial test loss:', adv_loss)

#     state = {
#         'net': net.state_dict()
#     }
#     if not os.path.isdir('checkpoint'):
#         os.mkdir('checkpoint')
#     torch.save(state, './checkpoint/' + file_name)
#     print('Model Saved!')

# def adjust_learning_rate(optimizer, epoch):
#     lr = learning_rate
#     if epoch >= 100:
#         lr /= 10
#     if epoch >= 150:
#         lr /= 10
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

# print('net_basic attacked with PGD attack: ')
# for epoch in range(0, 1):
#     adjust_learning_rate(optimizer, epoch)
#     # train(epoch)
#     test(epoch)

ModuleNotFoundError: No module named 'models'

In [ ]:
# from numpy import copy
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# import torch.backends.cudnn as cudnn

# import torchvision
# import torchvision.transforms as transforms

# import os


# learning_rate = 0.1
# epsilon = 0.0314
# k = 7
# alpha = 0.00784
# file_name = 'pgd_adversarial_training'

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
# ])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
# ])

# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

# class LinfPGDAttack(object):
#     def __init__(self, model):
#         self.model = model

#     def perturb(self, x_natural, y):
#         x = x_natural.detach()
#         x = x + torch.zeros_like(x).uniform_(-epsilon, epsilon)
#         for i in range(k):
#             x.requires_grad_()
#             with torch.enable_grad():
#                 logits = self.model(x)
#                 loss = F.cross_entropy(logits, y)
#             grad = torch.autograd.grad(loss, [x])[0]
#             x = x.detach() + alpha * torch.sign(grad.detach())
#             x = torch.min(torch.max(x, x_natural - epsilon), x_natural + epsilon)
#             x = torch.clamp(x, 0, 1)
#         return x

# def attack(x, y, model, adversary):
#     model_copied = copy.deepcopy(model)
#     model_copied.eval()
#     adversary.model = model_copied
#     adv = adversary.perturb(x, y)
#     return adv

# net = net_fgsm

# adversary = LinfPGDAttack(net)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

# def train(epoch):
#     print('\n[ Train epoch: %d ]' % epoch)
#     net.train()
#     train_loss = 0
#     correct = 0
#     total = 0
#     for batch_idx, (inputs, targets) in enumerate(train_loader):
#         inputs, targets = inputs.to(device), targets.to(device)
#         optimizer.zero_grad()

#         adv = adversary.perturb(inputs, targets)
#         adv_outputs = net(adv)
#         loss = criterion(adv_outputs, targets)
#         loss.backward()

#         optimizer.step()
#         train_loss += loss.item()
#         _, predicted = adv_outputs.max(1)

#         total += targets.size(0)
#         correct += predicted.eq(targets).sum().item()
        
#         if batch_idx % 10 == 0:
#             print('\nCurrent batch:', str(batch_idx))
#             print('Current adversarial train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
#             print('Current adversarial train loss:', loss.item())

#     print('\nTotal adversarial train accuarcy:', 100. * correct / total)
#     print('Total adversarial train loss:', train_loss)

# def test(epoch):
#     print('\n[ Test epoch: %d ]' % epoch)
#     net.eval()
#     benign_loss = 0
#     adv_loss = 0
#     benign_correct = 0
#     adv_correct = 0
#     total = 0
#     with torch.no_grad():
#         for batch_idx, (inputs, targets) in enumerate(test_loader):
#             inputs, targets = inputs.to(device), targets.to(device)
#             total += targets.size(0)

#             outputs = net(inputs)
#             loss = criterion(outputs, targets)
#             benign_loss += loss.item()

#             _, predicted = outputs.max(1)
#             benign_correct += predicted.eq(targets).sum().item()

#             if batch_idx % 10 == 0:
#                 print('\nCurrent batch:', str(batch_idx))
#                 print('Current benign test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
#                 print('Current benign test loss:', loss.item())

#             adv = adversary.perturb(inputs, targets)
#             adv_outputs = net(adv)
#             loss = criterion(adv_outputs, targets)
#             adv_loss += loss.item()

#             _, predicted = adv_outputs.max(1)
#             adv_correct += predicted.eq(targets).sum().item()

#             if batch_idx % 10 == 0:
#                 print('Current adversarial test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
#                 print('Current adversarial test loss:', loss.item())

#     print('\nTotal benign test accuarcy:', 100. * benign_correct / total)
#     print('Total adversarial test Accuarcy:', 100. * adv_correct / total)
#     print('Total benign test loss:', benign_loss)
#     print('Total adversarial test loss:', adv_loss)

#     state = {
#         'net': net.state_dict()
#     }
#     if not os.path.isdir('checkpoint'):
#         os.mkdir('checkpoint')
#     torch.save(state, './checkpoint/' + file_name)
#     print('Model Saved!')

# def adjust_learning_rate(optimizer, epoch):
#     lr = learning_rate
#     if epoch >= 100:
#         lr /= 10
#     if epoch >= 150:
#         lr /= 10
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

# print('net_fgsm attacked with PGD attack: ')
# for epoch in range(0, 1):
#     adjust_learning_rate(optimizer, epoch)
#     # train(epoch)
#     test(epoch)

In [ ]:
# from numpy import copy
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# import torch.backends.cudnn as cudnn

# import torchvision
# import torchvision.transforms as transforms

# import os

# learning_rate = 0.1
# epsilon = 0.0314
# k = 7
# alpha = 0.00784
# file_name = 'pgd_adversarial_training'

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
# ])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
# ])

# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

# class LinfPGDAttack(object):
#     def __init__(self, model):
#         self.model = model

#     def perturb(self, x_natural, y):
#         x = x_natural.detach()
#         x = x + torch.zeros_like(x).uniform_(-epsilon, epsilon)
#         for i in range(k):
#             x.requires_grad_()
#             with torch.enable_grad():
#                 logits = self.model(x)
#                 loss = F.cross_entropy(logits, y)
#             grad = torch.autograd.grad(loss, [x])[0]
#             x = x.detach() + alpha * torch.sign(grad.detach())
#             x = torch.min(torch.max(x, x_natural - epsilon), x_natural + epsilon)
#             x = torch.clamp(x, 0, 1)
#         return x

# def attack(x, y, model, adversary):
#     model_copied = copy.deepcopy(model)
#     model_copied.eval()
#     adversary.model = model_copied
#     adv = adversary.perturb(x, y)
#     return adv

# net = net_pgd

# adversary = LinfPGDAttack(net)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

# def train(epoch):
#     print('\n[ Train epoch: %d ]' % epoch)
#     net.train()
#     train_loss = 0
#     correct = 0
#     total = 0
#     for batch_idx, (inputs, targets) in enumerate(train_loader):
#         inputs, targets = inputs.to(device), targets.to(device)
#         optimizer.zero_grad()

#         adv = adversary.perturb(inputs, targets)
#         adv_outputs = net(adv)
#         loss = criterion(adv_outputs, targets)
#         loss.backward()

#         optimizer.step()
#         train_loss += loss.item()
#         _, predicted = adv_outputs.max(1)

#         total += targets.size(0)
#         correct += predicted.eq(targets).sum().item()
        
#         if batch_idx % 10 == 0:
#             print('\nCurrent batch:', str(batch_idx))
#             print('Current adversarial train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
#             print('Current adversarial train loss:', loss.item())

#     print('\nTotal adversarial train accuarcy:', 100. * correct / total)
#     print('Total adversarial train loss:', train_loss)

# def test(epoch):
#     print('\n[ Test epoch: %d ]' % epoch)
#     net.eval()
#     benign_loss = 0
#     adv_loss = 0
#     benign_correct = 0
#     adv_correct = 0
#     total = 0
#     with torch.no_grad():
#         for batch_idx, (inputs, targets) in enumerate(test_loader):
#             inputs, targets = inputs.to(device), targets.to(device)
#             total += targets.size(0)

#             outputs = net(inputs)
#             loss = criterion(outputs, targets)
#             benign_loss += loss.item()

#             _, predicted = outputs.max(1)
#             benign_correct += predicted.eq(targets).sum().item()

#             if batch_idx % 10 == 0:
#                 print('\nCurrent batch:', str(batch_idx))
#                 print('Current benign test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
#                 print('Current benign test loss:', loss.item())

#             adv = adversary.perturb(inputs, targets)
#             adv_outputs = net(adv)
#             loss = criterion(adv_outputs, targets)
#             adv_loss += loss.item()

#             _, predicted = adv_outputs.max(1)
#             adv_correct += predicted.eq(targets).sum().item()

#             if batch_idx % 10 == 0:
#                 print('Current adversarial test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
#                 print('Current adversarial test loss:', loss.item())

#     print('\nTotal benign test accuarcy:', 100. * benign_correct / total)
#     print('Total adversarial test Accuarcy:', 100. * adv_correct / total)
#     print('Total benign test loss:', benign_loss)
#     print('Total adversarial test loss:', adv_loss)

#     state = {
#         'net': net.state_dict()
#     }
#     if not os.path.isdir('checkpoint'):
#         os.mkdir('checkpoint')
#     torch.save(state, './checkpoint/' + file_name)
#     print('Model Saved!')

# def adjust_learning_rate(optimizer, epoch):
#     lr = learning_rate
#     if epoch >= 100:
#         lr /= 10
#     if epoch >= 150:
#         lr /= 10
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

# print('net_pgd attacked with PGD attack: ')
# for epoch in range(0, 1):
#     adjust_learning_rate(optimizer, epoch)
#     # train(epoch)
#     test(epoch)

In [ ]:
# import torch
# import torchvision
# import torchvision.transforms as transforms
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# import torchattacks

# # Define the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load the pretrained ResNet18 model
# model = torchvision.models.resnet18(pretrained=True)
# model = model.to(device)
# model.eval()

# # Define the CIFAR-10 transforms
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])

# # Load the CIFAR-10 test dataset
# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# # Define the PGD attack
# epsilon = 8/255  # Perturbation
# alpha = 2/255    # Step size
# steps = 12        # Number of iterations

# pgd = torchattacks.PGD(model, eps=epsilon, alpha=alpha, steps=steps)

# # Function to test the model on adversarial examples
# def test_model_on_adversarial(loader, model, attack):
#     correct = 0
#     total = 0
#     model.eval()
    
#     for data in loader:
#         images, labels = data
#         images, labels = images.to(device), labels.to(device)

#         # Generate adversarial examples
#         adv_images = attack(images, labels)

#         # Forward pass the adversarial examples through the model
#         outputs = model(adv_images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# # Test the model on the adversarially-attacked CIFAR-10 test set
# test_model_on_adversarial(testloader, model, pgd)


/home/pratyush/miniconda3/envs/pytorch_3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Accuracy of the model on adversarial examples: 0.00%


In [14]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchattacks

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained ResNet18 model
model = net_chaosLoss
model.eval()

# Define the CIFAR-10 transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define the PGD attack
epsilon = 8/255  # Perturbation
alpha = 2/255    # Step size
steps = 20        # Number of iterations

pgd = torchattacks.PGD(model, eps=epsilon, alpha=alpha, steps=steps)

# Function to test the model on adversarial examples
def test_model_on_adversarial(loader, model, attack):
    correct = 0
    total = 0
    model.eval()
    
    for data in loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # Generate adversarial examples
        adv_images = attack(images, labels)

        # Forward pass the adversarial examples through the model
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# Test the model on the adversarially-attacked CIFAR-10 test set
test_model_on_adversarial(testloader, model, pgd)


Files already downloaded and verified


In [ ]:
# import torch
# import torchvision
# import torchvision.transforms as transforms
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# import torchattacks

# # Define the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load the pretrained ResNet18 model
# model = net_basic
# model.eval()

# # Define the CIFAR-10 transforms
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])

# # Load the CIFAR-10 test dataset
# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# # Define the PGD attack
# epsilon = 60/255  # Perturbation
# alpha = 2/255    # Step size
# steps = 20        # Number of iterations

# pgd = torchattacks.PGD(model, eps=epsilon, alpha=alpha, steps=steps, norm='l2')

# # Function to test the model on adversarial examples
# def test_model_on_adversarial(loader, model, attack):
#     correct = 0
#     total = 0
#     model.eval()
    
#     for data in loader:
#         images, labels = data
#         images, labels = images.to(device), labels.to(device)

#         # Generate adversarial examples
#         adv_images = attack(images, labels)

#         # Forward pass the adversarial examples through the model
#         outputs = model(adv_images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# # Test the model on the adversarially-attacked CIFAR-10 test set
# test_model_on_adversarial(testloader, model, pgd)


In [25]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchattacks

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained ResNet18 model
model = net_chaosLoss
model.eval()

# Define the CIFAR-10 transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define the FGSM attack
epsilon = 8/255  # Perturbation

fgsm = torchattacks.FGSM(model, eps=epsilon)

# Function to test the model on adversarial examples
def test_model_on_adversarial(loader, model, attack):
    correct = 0
    total = 0
    model.eval()
    
    for data in loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # Generate adversarial examples
        adv_images = attack(images, labels)

        # Forward pass the adversarial examples through the model
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# Test the model on the adversarially-attacked CIFAR-10 test set
test_model_on_adversarial(testloader, model, fgsm)


Files already downloaded and verified
Accuracy of the model on adversarial examples: 48.79%


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchattacks

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained ResNet18 model
model = net_fgsm
model.eval()

# Define the CIFAR-10 transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define the FGSM attack
epsilon = 8/255  # Perturbation

fgsm = torchattacks.FGSM(model, eps=epsilon)

# Function to test the model on adversarial examples
def test_model_on_adversarial(loader, model, attack):
    correct = 0
    total = 0
    model.eval()
    
    for data in loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # Generate adversarial examples
        adv_images = attack(images, labels)

        # Forward pass the adversarial examples through the model
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# Test the model on the adversarially-attacked CIFAR-10 test set
test_model_on_adversarial(testloader, model, fgsm)


In [23]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchattacks

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained ResNet18 model
model = net_chaosLoss
model.eval()

# Define the CIFAR-10 transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define the FGSM attack
epsilon = 8/255  # Perturbation

fgsm = torchattacks.FGSM(model, eps=epsilon)

# Function to test the model on adversarial examples
def test_model_on_adversarial(loader, model, attack):
    correct = 0
    total = 0
    model.eval()
    
    for data in loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # Generate adversarial examples
        adv_images = attack(images, labels)

        # Forward pass the adversarial examples through the model
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# Test the model on the adversarially-attacked CIFAR-10 test set
test_model_on_adversarial(testloader, model, fgsm)


Files already downloaded and verified
Accuracy of the model on adversarial examples: 43.32%


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchattacks

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained ResNet18 model
model = net_fgsm
model.eval()

# Define the CIFAR-10 transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define the PGD attack
epsilon = 8/255  # Perturbation
alpha = 2/255    # Step size
steps = 20        # Number of iterations

pgd = torchattacks.PGD(model, eps=epsilon, alpha=alpha, steps=steps)

# Function to test the model on adversarial examples
def test_model_on_adversarial(loader, model, attack):
    correct = 0
    total = 0
    model.eval()
    
    for data in loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # Generate adversarial examples
        adv_images = attack(images, labels)

        # Forward pass the adversarial examples through the model
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# Test the model on the adversarially-attacked CIFAR-10 test set
test_model_on_adversarial(testloader, model, pgd)


In [ ]:
# import torch
# import torchvision
# import torchvision.transforms as transforms
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# import torchattacks

# # Define the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load the pretrained ResNet18 model
# model = net_fgsm
# model.eval()

# # Define the CIFAR-10 transforms
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])

# # Load the CIFAR-10 test dataset
# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# # Define the PGD attack
# epsilon = 60/255  # Perturbation
# alpha = 2/255    # Step size
# steps = 20        # Number of iterations

# pgd = torchattacks.PGD(model, eps=epsilon, alpha=alpha, steps=steps, norm='l2')

# # Function to test the model on adversarial examples
# def test_model_on_adversarial(loader, model, attack):
#     correct = 0
#     total = 0
#     model.eval()
    
#     for data in loader:
#         images, labels = data
#         images, labels = images.to(device), labels.to(device)

#         # Generate adversarial examples
#         adv_images = attack(images, labels)

#         # Forward pass the adversarial examples through the model
#         outputs = model(adv_images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# # Test the model on the adversarially-attacked CIFAR-10 test set
# test_model_on_adversarial(testloader, model, pgd)


In [ ]:
# import torch
# import torchvision
# import torchvision.transforms as transforms
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# import torchattacks

# # Define the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load the pretrained ResNet18 model
# model = net_pgd
# model.eval()

# # Define the CIFAR-10 transforms
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])

# # Load the CIFAR-10 test dataset
# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# # Define the PGD attack
# epsilon = 60/255  # Perturbation
# alpha = 2/255    # Step size
# steps = 20        # Number of iterations

# pgd = torchattacks.PGD(model, eps=epsilon, alpha=alpha, steps=steps, norm='l2')

# # Function to test the model on adversarial examples
# def test_model_on_adversarial(loader, model, attack):
#     correct = 0
#     total = 0
#     model.eval()
    
#     for data in loader:
#         images, labels = data
#         images, labels = images.to(device), labels.to(device)

#         # Generate adversarial examples
#         adv_images = attack(images, labels)

#         # Forward pass the adversarial examples through the model
#         outputs = model(adv_images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# # Test the model on the adversarially-attacked CIFAR-10 test set
# test_model_on_adversarial(testloader, model, pgd)


In [21]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchattacks

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained ResNet18 model
model = net_pgd
model.eval()

# Define the CIFAR-10 transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define the PGD attack
epsilon = 8/255  # Perturbation
alpha = 2/255    # Step size
steps = 20        # Number of iterations

pgd = torchattacks.PGD(model, eps=epsilon, alpha=alpha, steps=steps)

# Function to test the model on adversarial examples
def test_model_on_adversarial(loader, model, attack):
    correct = 0
    total = 0
    model.eval()
    
    for data in loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # Generate adversarial examples
        adv_images = attack(images, labels)

        # Forward pass the adversarial examples through the model
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on adversarial examples: {100 * correct / total:.2f}%')

# Test the model on the adversarially-attacked CIFAR-10 test set
test_model_on_adversarial(testloader, model, pgd)


Files already downloaded and verified
Accuracy of the model on adversarial examples: 52.85%
